# Introduction to Neural Networks (NN)

In this notebook, we will begin by installing and importing the required libraries for our exploration of Neural Networks (NN). Before we delve into the intricacies of training a NN, we will first explore various activation functions, which play a crucial role in determining the output of each neuron in the network.

**NB!**
    📂 Note:
    To run this notebook successfully, make sure you download the required data files from the GitHub repository: https://github.com/lmoranglez/SummerSchool.git and save them in the same folder as this notebook.


In [ ]:
# run this cell once to install the required libraries
#import sys
#!{sys.executable} -m pip install pandas==2.2.3 scikit-learn==1.6.1 matplotlib==3.10.1 seaborn==0.13.2

In [ ]:
# Libraries to import
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Part 1: Understanding Activation Functions

Neural Networks are powerful ML models. A key component of NNs is the **activation function**, which introduces non-linearity, enabling the network to learn complex patterns.

In this Part 1, we will:
- Explore common activation functions in NNs.
- Visualize how they transform input values.

As shown in the presentation, the dot product of X·W plus b (features·weights + bias) results in a constant value, ressembling a multiple linear regression. 
$$
  v =  w_1x_1 +  w_2x_2 + ... + w_mx_m + b = \sum^m_{i=0} w_ix_i + b
$$
However, NN may require non-linearity to model complex patterns. This is where the **activation function** comes into play. By applying an **activation function** to $v$, we break linearity and enable the network to learn beyond simple linear relationships.

In [ ]:
# Define a linear space
x = np.linspace(-10, 10, 50)

In [ ]:
# Define activation functions
def sigmoid(x): return 1 / (1 + np.exp(-x))
def relu(x): return np.maximum(0, x)
def leaky_relu(x, alpha=0.1): return np.where(x > 0, x, alpha*x)
def tanh(x): return np.tanh(x)
def softplus(x): return np.log(1 + np.exp(x))

In [ ]:
# Select a value for v between -10 and 10 to visualize the transformation with the activation functions
v = 4  # Example pre-activation value (dot product + bias)
print(f'Changes when the activation functions are applied over v : {v}')

In [ ]:
# List of (function, color, name) for each activation
activations = [
    (sigmoid, 'blue', 'Sigmoid'),
    (relu, 'green', 'ReLU'),
    (leaky_relu, 'orange', 'Leaky ReLU'),
    (tanh, 'purple', 'Tanh'),
    (softplus, 'brown', 'Softplus')
]

# Plot settings
plt.figure(figsize=(15, 8))
plt.suptitle("Activation Functions", fontsize=16, y=1.02)

# Original data (linear transformation)
plt.subplot(2, 3, 1)
plt.scatter(x, x, color='red', label="Linear (No Activation)")
plt.axvline(v, color='black', linestyle='--', alpha=0.5)
plt.axhline(v, color='black', linestyle='--', alpha=0.5)
plt.scatter(v, v, color='black', s=100, label=f"v = {v}")
plt.title("Linear Transformation")
plt.grid()
plt.legend()

# Plot each activation
for i, (func, color, name) in enumerate(activations, start=2):
    plt.subplot(2, 3, i)
    plt.plot(x, func(x), color=color, label=name)
    plt.scatter(x, func(x), color=color, s=10)
    plt.axvline(v, color='black', linestyle='--', alpha=0.5)
    plt.axhline(func(v), color='black', linestyle='--', alpha=0.5)
    plt.scatter(v, func(v), color='black', s=100, label=f"{name}(v) = {func(v):.2f}")
    plt.title(f"{name} Activation")
    plt.grid()
    plt.legend()

plt.tight_layout()
plt.show()

**Experimentation**: Adjust the value of `v` (between -10 and 10) and re-run the cells to observe how each activation function transforms it.

# Part 2: Training a NN with Scikit-Learn

Now that we understand activation functions, let’s train a vanilla NN, a Multi-Layer Perceptron (MLP), using `scikit-learn` library.

Steps to follow:

    2.1. Load and get familiar with the dataset
    2.2. Preprocess the dataset
    2.3. Split data into input features (X) and target properties (y)
    2.4. Standarize the data set.
    2.5. Define the NN architecture and its hyperparameters
    2.6. Train the NN model
    2.7. Analyze the model performance
    2.8. Check for outliers
    2.9. Assess the evolution of the training procedure

**Goal** \
Predict a target property—either activation energies, bond distances, or angles—for derivatives of Vaska's complex (trans-[Ir(CO)Cl(PPh₃)₂]) using computed chemical descriptors as input features to train predictive models.

### 2.1. Load and get familiar with the dataset

**Dataset** 

- Key features (Input, $X$): electronic structure parameters (e.g., orbital energies, partial charges), distances, atomic numbers...
- Target variables (Output, $y$): 1) energy barriers (ΔE‡, in kcal/mol) for oxidative addition reactions; 2) H···H bond distances (d(H···H) in Å); 3) angles (angle(H-Ir-H) in º) of the dihydrogen activation in the Vaska's complexes (published in 2020, 2024).
[data_Vaska](https://github.com/uiocompcat/AABBA/blob/main/data_Vaska.zip)

In [ ]:
# Read data from the .csv file
df = pd.read_csv("NN_Vaskawithoutlier.csv")
df

In [ ]:
# Visualize target properties
distance = df['target_distance'].values
barrier = df['target_barrier'].values
angle = df['target_angle'].values

plt.figure(figsize=(12, 5))

plt.subplot(1, 3, 1)    # nrows, ncols, index
plt.hist(barrier, bins=20, color='purple', alpha=0.7)
plt.title('Barrier Distribution')
plt.xlabel('E (kcal/mol)')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)    
plt.hist(distance, bins=20, color='skyblue', alpha=0.7)
plt.title('H···H Distance Distribution')
plt.xlabel('d(H···H) (Å)')
plt.ylabel('Frequency')

plt.subplot(1, 3, 3)
plt.hist(angle, bins=20, color='grey', alpha=0.7)
plt.title('H···Ir···H Angle Distribution')
plt.xlabel('θ (degrees)')
plt.ylabel('Frequency')

# Mean and standard deviation of the target properties
print(f"Mean barrier: {np.mean(barrier):.2f} ± {np.std(barrier):.2f} kcal/mol")
print(f"Mean H···H distance: {np.mean(distance):.2f} ± {np.std(distance):.2f} Å")
print(f"Mean H···Ir···H angle: {np.mean(angle):.2f} ± {np.std(angle):.2f} degrees")

### 2.2. Preprocess the data set

**NB!** Data treatment is one of the most demanding aspects when applying ML. It is important to be mindful of the `type` of input we introduce. Always refer to the documentation webpage to understand the input data format required in the functions.

Let's split first the data into input features ($X$) and target feature ($y$). 
The three target properties $y$s were already separated as `barrier`, `distance` and `angle`.

In [ ]:
# Display all columns in the dataframe
print(df.columns.tolist())

In [ ]:
# Remove the id and target features from the df to define X
df_features = df.drop(columns=['id',
                            'target_distance', 
                            'target_barrier', 
                            'target_angle'])
X = df_features.values

# NB! Be sure that the X, y are a sequence of indexables with the same length / shape[0] = nrows
print('Track the size of the data set', 
      'X input:', X.shape, 
      'distance:', distance.shape, 'barrier:', barrier.shape, 'angle:', angle.shape)

### 2.3. Split data into input features (X) and target properties (y)

Split the data into train and test subsets. \
**NB!** In this implementation, validation is handled internally by scikit-learn and thus, no explicit split is needed.
Function to use: [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [ ]:
# Define the target property, the partition of the test_size and the seed to initialize the weigths and ensure reproducibility
target_property = barrier
test_percentage = 0.2
seed = 42

In [ ]:
# Split array into random train and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    target_property, 
                                                    test_size=test_percentage, 
                                                    random_state=seed)
print('Track the size of the data set', 
      'X train:', X_train.shape,
      'X test:', X_test.shape, 
      'y train:', y_train.shape, 
      'y test:',y_test.shape)

### 2.4. Standarize the data set

Common approach is to standarize the dataset:
$$
z = (x-u) / s
$$
where $u$ is the mean, and $s$ the standard deviation \
**NB!** To prevent data leakage, we should only fit the scaler on the training data and then transform the test data with train_mean and train_std

In [ ]:
scaler = StandardScaler()   # class 'StandardScaler' creates the instance/object 'scaler'
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### 2.5. Define the NN architecture and its hyperparameters 

Create a the NN architecture using [MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html) model. We must carefully first consider the basic building blocks of the architecture.

In [ ]:
# help(MLPRegressor)

In [ ]:
hidden_layer_sizes = (128, 128) 
activation = 'relu'
solver = 'adam'
epochs = 200
learning_rate = 'constant'
learning_rate_init=0.0001
random_state=2025
validation_fraction=0.1
early_stopping=True
verbose=False

In [ ]:
# Let’s begin by specifying our initial hyperparameters and their respective values.
mlp = MLPRegressor(
    hidden_layer_sizes=hidden_layer_sizes,  # number of neurons in the hidden layers   n of hidden layers 
    activation=activation, # activation function
    solver=solver,  # optimization algorithm
    max_iter=epochs,  # number of iterations (epochs) to train the model
    learning_rate=learning_rate,  # learning rate type
    learning_rate_init=learning_rate_init,  # initial learning rate
    random_state=random_state,  # random seed for reproducibility
    validation_fraction=validation_fraction, # by default
    early_stopping=early_stopping,  # stop training when validation score is not improving
    verbose=verbose,  # print progress messages
)

### 2.6. Train the NN model

In [ ]:
# Once the architecture is defined, the training can start
mlp.fit(X_train_scaled, y_train)   # X: input features, y: target property 

### 2.7 Analyze the model performance

There are two important aspects to consider: 1) the evolution of the loss function during the training and 2) the model performance.

In [ ]:
# 1) Evolution of the loss function 
loss_curve = mlp.loss_curve_  # get the loss curve (training error) for each iteration

fig, ax1 = plt.subplots(figsize=(6, 4))
plt.title('Training (and Validation Loss)')
plt.xlabel('Iteration')
# Plotting the training loss on the first axis
ax1.plot(loss_curve, label='Training Loss', color='blue')
ax1.set_ylabel('Loss', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.legend(loc='upper left')

# Add validation score (only if early_stopping=True)
if mlp.early_stopping:
    ax2 = ax1.twinx()
    ax2.plot(mlp.validation_scores_, label='Validation R²', color='orange')
    ax2.set_ylabel('Validation Score (R²)', color='orange')
    ax2.tick_params(axis='y', labelcolor='orange')
    ax2.legend(loc='center right')
    print(f"Final Validation R²: {mlp.validation_scores_[-1]:.3f}")
else:
    print("Validation scores not tracked (early_stopping=False)")

plt.tight_layout()
plt.show()

In [ ]:
# 2) Evaluate the model performance
train_score = mlp.score(X_train_scaled, y_train)
test_score = mlp.score(X_test_scaled, y_test)
y_train_pred = mlp.predict(X_train_scaled)
y_test_pred = mlp.predict(X_test_scaled)
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print(f"Final train performance, Train R²: {train_score:.3f} | Train MAE: {train_mae:.3f}")
print(f"Final test performance, Test R²: {test_score:.3f} | Test MAE: {test_mae:.3f}")

if mlp.early_stopping:
    validation_score = mlp.validation_scores_  # get the validation score for each iteration
    print(f"Final validation performance, Val. R²: {np.round(validation_score[-1], 3)}")

In [ ]:
def r2_score_manual(y, y_pred):    
    R2 = 1 - ((np.sum((y - y_pred) ** 2))/(np.sum((y - np.mean(y)) ** 2)))
    return R2

# Printing the predictions than with the training and test sets.
def plot_predictions(y_train_pred, y_train, y_test_pred, y_test):
    
    min_value_train = np.min(y_train_pred) - 1
    max_value_train = np.max(y_train_pred) + 1
    min_value_test = np.min(y_test_pred) - 1
    max_value_test = np.max(y_test_pred) + 1

    plt.figure(figsize=(12, 5))
    # Training data plot
    plt.subplot(1, 2, 1)  # Define a proper 1x2 grid of subplots
    plt.scatter(y_train_pred, y_train, c='blue', label='Training Data')
    plt.plot([min_value_train, max_value_train], [min_value_train, max_value_train], "r--")
    plt.text(min_value_train*5, min_value_train*1.2, f'R² = {r2_score_manual(y_train, y_train_pred):.2f}', fontsize=12, color='black')
    plt.xlabel('Predicted train values')
    plt.ylabel('True train values')
    plt.title('Training Prediction Performance')
    plt.legend()

    plt.subplot(1, 2, 2)  # Change to the correct subplot index
    plt.scatter(y_test_pred, y_test, c='#66c2a5', label='Test Data')
    plt.plot([min_value_test, max_value_test], [min_value_test, max_value_test], "r--")
    plt.text(min_value_test*5, min_value_test*1.2, f'R² = {r2_score_manual(y_test, y_test_pred):.2f}', fontsize=12, color='black')
    plt.xlabel('Predicted test values')
    plt.ylabel('True test values')
    plt.title('Test Prediction Performance')
    plt.legend()

    # Set same length axis ranges with 5% margin of max value
    plt.subplots_adjust(wspace=0.3)  # Adjust spacing between subplots

    print(f'Train R²= {r2_score_manual(y_train, y_train_pred):.2f}, ', f'Test R²= {r2_score_manual(y_test, y_test_pred):.2f}')

    min_value_train, max_value_train, min_value_test, max_value_test = 0,0,0,0
    return fig

In [ ]:
fig = plot_predictions(y_train_pred, y_train, y_test_pred, y_test)
plt.show()

### 2.8. Check for outliers

It is crucial to visualize the results, as the presence of outliers can sometimes create a misleading impression of poor model training and subpar performance. The model may actually be functioning well, but outliers can obscure this reality in various evaluation metrics.

In [ ]:
def identify_outlier(pred_list):
    mean_pred = np.mean(pred_list)
    threshold = 10 if np.array_equal(target_property, distance) else 50
    for index, item in enumerate(pred_list):
        if item < -mean_pred * threshold or item > mean_pred * threshold:
            return mean_pred, item, index
    return None, None, None  # Explicit return if no outl

mean_pred, outlier, id_outlier = identify_outlier(y_test_pred)

if outlier is not None and id_outlier is not None: 
    print('Target property mean value:', np.round(mean_pred, 2),
          'Outlier value:', np.round(outlier, 3), 
          'Outlier index value:', id_outlier)
    
    #if outlier and id_outliter are not (None, None): 
    y_test_pred = np.delete(y_test_pred, id_outlier)
    y_test = np.delete(y_test, id_outlier)
    X_test_scaled = np.delete(X_test_scaled, id_outlier, axis=0)

index = 0 
item = 0

In [ ]:
fig_test = plot_predictions(y_train_pred, y_train, y_test_pred, y_test)
plt.show()

### 2.9. Assess the evolution of the training procedure

The MLPRegressor in `scikit-learn` does not provide performance tracking during each epoch. One alternative is to save model after each epoch, allowing you to load and evaluate it later. This way, you can analyze its performance evolution at each epoch.

In [ ]:
model = MLPRegressor(
    hidden_layer_sizes=hidden_layer_sizes,  # number of neurons in the hidden layers   n of hidden layers 
    activation=activation, # activation function
    solver=solver,  # optimization algorithm
    learning_rate=learning_rate,  # learning rate type
    learning_rate_init=learning_rate_init,  # initial learning rate
    random_state=random_state,  # random seed for reproducibility
    validation_fraction=validation_fraction, # by default
    early_stopping=early_stopping,  # stop training when validation score is not improving
    
    warm_start= True,
    max_iter=1,  # number of iterations (epochs) to train the model
)

In [ ]:
train_r2 = []
test_r2 = []
train_mae = []
test_mae = []

import warnings
from sklearn.exceptions import ConvergenceWarning

# Disable ConvergenceWarning for the loop
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    epochs = 200
    for epoch in range(epochs):
        model.fit(X_train_scaled, y_train)  # Trains 1 epoch (due to max_iter=1)
    
        # Predictions
        y_train_pred = model.predict(X_train_scaled)
        y_test_pred = model.predict(X_test_scaled)
    
        # Log metrics
        train_r2.append(r2_score(y_train, y_train_pred))
        test_r2.append(r2_score(y_test, y_test_pred))
        train_mae.append(mean_absolute_error(y_train, y_train_pred))
        test_mae.append(mean_absolute_error(y_test, y_test_pred))
    
        # Print progress
        if epoch % 10 == 0:
            print(f"Epoch {epoch}:")
            print(f"  Train R²: {train_r2[-1]:.3f} | Test R²: {test_r2[-1]:.3f}")
            print(f"  Train MAE: {train_mae[-1]:.3f} | Test MAE: {test_mae[-1]:.3f}")
            print("---")

In [ ]:
# After the training loop
def plot_metrics(train_r2, test_r2, train_mae, test_mae):
    epochs = range(len(train_r2))  # Since you are logging every epoch

    # Create subplots
    plt.figure(figsize=(10, 5))
    # Training data plot
    plt.subplot(1, 2, 1)  # Define a proper 1x2 grid of subplots

    # Plotting R² scores
    plt.plot(epochs, train_r2, label='Train R²', color='blue', marker='o', ms=4,  alpha=0.6)
    plt.plot(epochs, test_r2, label='Test R²', color='#66c2a5', marker='o', ms=4,  alpha=0.6)
    plt.title('R² Scores over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('R² Score')
    plt.legend()
    plt.grid()

    # Plotting MAE
    plt.subplot(1, 2, 2)  # Define a proper 1x2 grid of subplots
    plt.plot(epochs, train_mae, label='Train MAE', color='blue', linestyle='--', marker='o', ms=4, alpha=0.6)
    plt.plot(epochs, test_mae, label='Test MAE', color='#66c2a5', linestyle='--', marker='o', ms=4, alpha=0.6)
    plt.title('MAE over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('MAE')
    plt.legend()
    plt.grid()
    plt.tight_layout()
    
    plt.subplots_adjust(wspace=0.3)  # Adjust spacing between subplots

    return fig 

In [ ]:
# Call the plotting function after fitting the model
fig = plot_metrics(train_r2, test_r2, train_mae, test_mae)
plt.show()

### 2.10. Next Steps: Experimentation Ideas
Now that you have trained your model, consider trying the following to enhance your exploration:

1. **Data Preparation**:
    In the **2.3** section, you can explore:
    - Trying augmenting ordecreasing your training data. Adjust `test_percentage = 0.2`
    - Changing the shuffle seed. `seed = 42`
    - Running the model with a different target property.  You can set `target_property =` `barrier` OR `distance` OR `angle`
    
2. **Hyperparameter Tuning**:
    In the **2.5** section, consider:
   - Adjusting the learning rate `learning_rate_init=0.0001`. Experiment with learning_rate_init values ranging from 0.0001 to 0.001, and observe in section 2.8. how the training curves change.
   - Modifying the number of epochs based on the results observed in section 2.8.
   - Implementing learning rate schedules for optimization. Change `learning_rate = 'constant'` to a different strategy. 
   - Modifying the initiallization seed for the model. `random_state=2025`

3. **Model Architecture Adjustments**:
    In the **2.5** section, try the following:
   - Adding more hidden layers, but be mindful of potential overfitting due to increased parameters.
   - Experimenting with different activation functions.

4. **Document Your Findings**:
   - Keep notes on different configurations and their impacts on model performance!

Important Note:

Remember that the Jupyter Notebook retains variable values in memory. If you encounter any unexpected results or "weird" values, it’s best to rerun the entire page to refresh the environment.

Feel free to modify the code in the cells above and re-run the training block. Happy experimenting!